In [9]:
!wget https://s3-us-west-1.amazonaws.com/restaurant-review-data/kaggle/whats-cooking/train.json.zip
!unzip train.json.zip

--2018-04-24 02:54:57--  https://s3-us-west-1.amazonaws.com/restaurant-review-data/kaggle/whats-cooking/train.json.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.20.105
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.20.105|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1845320 (1.8M) [application/zip]
Saving to: ‘train.json.zip’

train.json.zip      100%[===================>]   1.76M  3.76MB/s    in 0.5s    

2018-04-24 02:54:57 (3.76 MB/s) - ‘train.json.zip’ saved [1845320/1845320]



In [94]:
import re
from functools import partial
import json
with open('train.json') as f:
    whatscooking = json.load(f)
    ingredients = [i['ingredients'] for i in whatscooking]
    
ingredients = [ingredient for ingredientlist in ingredients for ingredient in ingredientlist]
ingredients = map(str.lower, ingredients)

clean_up_strings = [
    '\(.+oz\.\)'
]
clean_up_patterns = [re.compile(p) for p in clean_up_strings]

def clean(patterns, string):
    for pattern in patterns:
        string = pattern.sub("", string)
    return string.strip()

clean = partial(clean, clean_up_patterns)

ingredients = map(clean, ingredients)
ingredients = sorted(list(set(ingredients)))


In [148]:
alphanumeric = re.compile("[^A-Za-z0-9]")

tokens = set()
for token in nlp(" ".join(ingredients)):
    p = alphanumeric.sub("", token.norm_)
    if len(p) > 1:
        tokens.add(p)
    
tokens = list(set(tokens))
pattern = re.compile(f"({'|'.join(tokens)})")


In [375]:
def yield_subchars(string):
    for i in range(len(string)):
        yield string[:i+1]
        
def to_string_list(doc_or_span):
    return [i.norm_ + i.whitespace_ for i in doc_or_span]

def yield_subtokens(doc):
    for i in range(len(doc)):
        yield doc[:i+1]
        
def yield_substring_lists(doc_or_span):
    for i in range(len(doc)):
        yield to_string_list(doc[:i+1])        

In [364]:
class Parse:
    def __init__(self, token_bank):
        self.token_bank = token_bank
        self.token_graph = self.build_token_graph()
        
    def build_token_graph(self):
        """
        refactor to pass over token bank just once you idiot.
        """
        g = nx.DiGraph()
        for doc in nlp.pipe(self.token_bank, disable=['ner','parser','tagger']):
            doclist = to_string_list(doc)
            docstring = ''.join(doclist)
            g.add_node(docstring)
        for doc in nlp.pipe(self.token_bank, disable=['ner','parser','tagger']):
            doclist = to_string_list(doc)
            docstring = ''.join(doclist)            
            for subdoc in yield_substring_lists(doc):
                substring = "".join(subdoc)
                if substring in g:
                    g.add_edge(substring, docstring)
        return g
    
    def _recurse_get(self, subdoc, key):
        
        if len(subdoc) == 1:
            if key:
                new_key = f"{key} {subdoc[0].norm_}"
            else:
                new_key = subdoc[0].norm_
            
            if new_key in self.token_graph:
                return None, new_key
            else:
                return None, None
            
            
        elif len(subdoc) > 1:    
            next_word = subdoc[0]
            subdoc = subdoc[1:]

            if key:
                new_key = f"{key} {next_word.norm_}"
            else:
                new_key = next_word.norm_

            if new_key in self.token_graph:
                subdoc, key = self._recurse_get(subdoc, new_key)
            else:
                pass
            return subdoc, key
        else:
            return None, None

            
        
    def parse(self, texts):
        for doc in nlp.pipe(texts, disable=['ner','parser','tagger']):
            ingredients = []
            while doc is not None:
                doc, matched_token = self._recurse_get(doc, "")
                if matched_token is not None and matched_token != "":
                    ingredients.append(matched_token)
            yield ingredients

    
            
        
        

In [366]:
p = Parse(ingredients)

In [390]:
i = 3
print(df.text.iloc[i])
print()
print(list(p.parse([df.text.iloc[i]])))

b"Lester's is located in a beautiful neighborhood and has been there since 1951. They are known for smoked meat which most deli's have but their brisket sandwich is what I come to montreal for. They've got about 12 seats outside to go along with the inside. \n\nThe smoked meat is up there in quality and taste with Schwartz's and you'll find less tourists at Lester's as well."

[['meat', 'brisket', 'meat']]


In [1]:
from dask.dataframe import read_csv
df = read_csv("../dataset/review.csv")

In [ ]:
print(len(df))

In [1]:
from pandas import read_csv
import spacy
nlp = spacy.load('en')
df = read_csv("../dataset/review.csv", nrows=100000)

In [7]:
doc = nlp(df.text.iloc[0])
ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
ents

[('around lunch or dinner hours', 73, 101, 'TIME'),
 ('half', 234, 238, 'CARDINAL'),
 ('Hand', 269, 273, 'GPE')]

In [229]:

bus = read_csv("../dataset/business.csv")

In [231]:
!ls ../dataset/

business.csv				 review.json
business.json				 tip.csv
checkin.csv				 tip.json
checkin.json				 user.csv
Dataset_Challenge_Dataset_Agreement.pdf  user.json
photos.json				 Yelp_Dataset_Challenge_Round_11.pdf
review.csv


In [389]:


class DocVisualizer:
    def __init__(self, text, nlp):
        self.doc = nlp(self.preprocess(text))
        
    @staticmethod
    def preprocess(text):
        return text.replace("\\n", "")
        
    def _build_token_tables(self):
        r = {}
        for i, token in enumerate(self.doc):
            r[i] = self._build_token_view(token)
        return r
    
    def _make_js_pop(self):
        d = self._build_token_tables()
        s = ""
        for key, value in d.items():
            s+= f'mydict[{key}] = `{value}`\n'
        return s
        
    def _build_token_view(self, token):
        tablestr = f"""
        <table style="width:100%">
          <tr>
            <th>Token</th>
            <th>POS</th> 
            <th>DEP</th>
            <th>head</th>            
          </tr>
          <tr>
            <td>{token.lemma_}</td>
            <td>{token.pos_}</td> 
            <td>{token.dep_}</td>
            <td>{token.head}</td>            
          </tr>"""
        return tablestr
        
    def _build_doc_view(self):
        
        raw = "".join(map(lambda token: f"<span id={token.i} onmouseover='highlight(this);' onmouseout='unhighlight(this)'>{token.string + token.whitespace_}</span>", self.doc))
        raw += f"""
        <div id='rdiv'>
        
        </div>
        <script>
        
        function unhighlight(x) {{
          x.style.backgroundColor = 'white'
        }}        
    

        function highlight(x) {{
          x.style.backgroundColor = 'red'
          var inner = mydict[x.id]
          div = document.getElementById("rdiv")
          div.innerHTML = inner
        }}
        
        var mydict = {{}}
        {self._make_js_pop()}
        </script>
        """
        return raw
        
        

In [388]:
text = df.text.iloc[2]
d = DocVisualizer(text, nlp)
from IPython.display import HTML, display
display(HTML(d._build_doc_view()))